In [1]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm
!pip install gtts
from gtts import gTTS
from io import BytesIO
from tempfile import TemporaryFile
import librosa
import IPython.display


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ewro6mhk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ewro6mhk
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Tesi/testtraduzioneSTT
!ls

/content/drive/MyDrive/Tesi/testtraduzioneSTT
audioRecord.wav  audioSpec.jpeg  dialogo.ipynb	provaditraduzione.ipynb


In [40]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

path_in = '/content/drive/MyDrive/Tesi/testtraduzioneSTT'

def record(sec=5):
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])

    with open('audioRecord.wav','wb') as f:
        f.write(b)
    return 'audioRecord.wav'  # or webm ?
    #audio = AudioSegment.from_file(BytesIO(b))
    #return audio


model = whisper.load_model("small")

In [43]:
#we have to import our neural network for emotion recognition
import os
import json
import tensorflow as tf
from tensorflow import keras
import re
from IPython.display import Audio
import matplotlib.pyplot as plt


#emotion_model_path = '/content/drive/MyDrive/Tesi/InceptionV3/InceptionV3.h5'
emotion_model_path = '/content/drive/MyDrive/Tesi/model/imageclassifier.h5'
emotion_model = keras.models.load_model(emotion_model_path)

#Dictionary with emotions
dicts = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fearful', 7:'disgust', 8:'surprised'}

# Path if the audio and spectrum files
path_load = 'audioRecord.wav'
path_save = 'audioSpec.jpeg'

img_height = 180
img_width = 180

image = tf.keras.utils.load_img('audioSpec.jpeg',target_size=(img_height, img_width))

In [56]:
# Record the audio
audio = record()


# make log-Mel spectrogram and move to the same device as the model
y, sr = librosa.load(path_load)
#yt, _ = librosa.effects.trim(y)
#y = yt
y, _ = librosa.effects.trim(y)
mel_spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=512)
mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=50000, x_axis='time')
plt.axis('off')
plt.savefig(path_save)
plt.close()

# Spectrum for the traslation
#audio = whisper.load_audio("audioRecord.wav")
#audio = whisper.pad_or_trim(audio)
#mel = whisper.log_mel_spectrogram(audio).to(model.device)
#options = whisper.DecodingOptions(language= 'it', fp16 = False)
#result = whisper.decode(model, mel, options)
result = model.transcribe('audioRecord.wav', language="it", fp16=False, verbose=False)
#print(result.text)
print(result)
#print(type(result.text))


#image = tf.keras.utils.load_img('audioSpec.jpeg',target_size=(img_height, img_width))
input_arr = tf.keras.utils.img_to_array(image)
input_arr = np.array([input_arr])# Convert single image to a batch.

#normalization
#max = np.max(input_arr)
#input_arr = input_arr/max
input_arr = np.divide(input_arr, np.max(input_arr))
print(input_arr.min()) # we check for instance the min
print(input_arr.max()) # we check for instance the max

predictions = emotion_model.predict(input_arr)
print("L'emozione carpita è: {}".format(dicts[np.argmax(predictions)]))

def compare_strings(string1: str, string2: str):
    pattern = re.compile(string2)
    match = re.search(pattern, string1)
    if match:
        return 1
    else:
        return 0

text = 'Vesuvio erutta, tutta Napoli è distrutta Porco Dio'
tts = gTTS(text=text, lang='it')
f = TemporaryFile()
tts.write_to_fp(f)

f.seek(0)
audio, sr = librosa.load(f)
IPython.display.Audio(audio, rate=sr)

<IPython.core.display.Javascript object>

<ipython-input-56-adfc91cb82c0>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path_load)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
100%|██████████| 534/534 [00:15<00:00, 35.52frames/s]


{'text': ' non ne posso più viennese fuori che sono sempre raviato che palle no basta', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.24, 'text': ' non ne posso più viennese fuori che sono sempre raviato che palle no basta', 'tokens': [50364, 2107, 408, 22501, 10589, 371, 1053, 77, 1130, 8536, 7386, 947, 9259, 9553, 367, 18442, 2513, 947, 280, 11780, 572, 45282, 50626], 'temperature': 0.0, 'avg_logprob': -0.5023831526438395, 'compression_ratio': 1.0869565217391304, 'no_speech_prob': 0.4903927147388458}], 'language': 'it'}
0.0
1.0
1/1 [==============================] - 0s 375ms/step
L'emozione carpita è: fearful


In [7]:
import re
audio = record()
#audio

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audioRecord.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
#mel_to_array = tf.keras.preprocessing.image.img_to_array(mel)
#desired_shape = (180, 180, 3)
#mel_resized = tf.image.resize(mel_to_array, size=(desired_shape[0], desired_shape[1]))
#mel_resized = tf.expand_dims(mel_resized, axis=0)  # Aggiungi una dimensione per il batch
#emotion = emotion_model.predict(mel_resized)

#print(f"L'emozione rilevata è: %s \n", emotion)
# decode the audio
options = whisper.DecodingOptions(fp16 = False)
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)
print(type(result.text))

def compare_strings(string1: str, string2: str):
    pattern = re.compile(string2)
    match = re.search(pattern, string1)

    if match:
        return 1
    else:
        return 0

phrase1 = "dove sono andati i ragazzi?"
phrase2 = "cosa si mangià per cena?"
if(compare_strings(result.text.lower(), phrase1)):
    text = 'Dove sono andati i ragazzi?'
elif(compare_strings(result.text.lower(), phrase2)):
    text = 'Cosa si mangia per cena?'
else:
    text = 'Feed from desire'
tts = gTTS(text=text, lang='it')
f = TemporaryFile()
tts.write_to_fp(f)

f.seek(0)
audio, sr = librosa.load(f)
IPython.display.Audio(audio, rate=sr)

<IPython.core.display.Javascript object>

Thank you.
<class 'str'>
